# Healthy Streets of Los Angeles Injuries/Deaths which could be prevented with implemented Mobility in plan 2022
This project finds a number of car accidents that could have been prevented if the Mobility plan was implemented earlier going back to 2015.


Sources:
* Injuries/Deaths in the City of LA https://data.lacity.org/Public-Safety/Traffic-Collision-Data-from-2010-to-Present/d5tf-ez2w
* Implepemented Mobility Plan shapefiles (see hsla_mobilty_plan project)

Assumptions:
* Location only City of Los Angeles
* years are 2015-2023
* Excluding freeways (STATE_ROUTE is Null)

Output:
* a number of deaths/injuries on LA streets that happened on streets missing mobility safety plan improvements at that time

Note:
* it doesn't matter if we overlap roads with implemented mobility measures only after 2015 or an general, because we're looking for preventable crashes. so it means that all crashes are after 2015, and they are preventable if there is no mobility plan measure has been implemented at the time of the crash, so if already have a bike path - the crash is not preventable.

Any questions - @sunchugasheva

In [ ]:
import pandas as pd
import datetime
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import folium

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

In [ ]:
today = datetime.datetime.now().strftime("%Y_%m_%d")
print(today)
year_mobility = 2015 # we compare only mobility plan improvements, so after 2015
radius = 20 # our average street will be 40m wide

## functions

In [ ]:
def col_from_code(col, code):
    new_col = col.str.contains('|'.join(code)).replace(
        {True: 1, False: 0}
    ).fillna(0).astype(int)
    
    return new_col

def get_coords(line):
    return list(map(float, line[1:-1].split(', ')))

In [ ]:
# todo: change all show_map to show_maps
def show_map(gdf1, name1, color1, gdf2, name2, color2):
    
    print(
            f'{color1}: {name1}, {color2}: {name2}'
        )
    
    f = folium.Figure(width=1000, height=500)
    
    m = gdf1.explore(
        name = name1,
        color = color1
    ).add_to(f)

    map_2 = gdf2.explore(
        m=m,  # pass the map object
        name = name2,
        color = color2
    )

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 

    folium.LayerControl().add_to(m)
    
    return m

def show_maps(gdf_names):
    '''
    show a number of gdfs with set colors
    - gdfs_colors - dictionary with format:{'name': gdf}
    ''' 
    colors = ['green', 'blue', 'red', 'orange', 'purple', 'yellow', 'magenta']
    f = folium.Figure(width=1000, height=500)
    m = folium.Map(location = [34.05, -118.24], zoom_start=50).add_to(f)
    i = 0
    
    for name in gdf_names.keys():
        color = colors[i]
        i += 1
        gdf = gdf_names[name]
        print(f'{color}: {name}')
        if gdf.loc[0, 'geometry'].geom_type!='Point':
            gdf.explore(
                m = m,
                name = name,
                color = color
            )
        else:
            #folium.Marker(gdf).add_to(m)
            #folium.features.GeoJson(gdf).add_to(m)
            folium.GeoJson(
                gdf,
                marker=folium.Circle(
                    radius=20, fill_color=color,
                    fill_opacity=0.6, color=color, weight=1
                )                
            ).add_to(m)  

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 
    folium.LayerControl().add_to(m)
    
    return m

In [ ]:
def buffer(gdf, radius, proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame, has column "geometry"
    - radius - radius of bufferm meters
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''  
    gdf = gdf.copy()
    gdf_calc = gdf.to_crs(proj_calc)
    #print('data proj:', proj, '\ncalculation proj: ', proj_calc)
    gdf['buffered'] = gdf_calc.buffer(radius, cap_style=2).to_crs(proj)

    gdf_return = gdf.copy()
    gdf_return.geometry = gdf_return.buffered
    gdf_return.set_geometry('geometry', inplace=True)
    gdf_return.geometry.crs = proj
    
    return gdf_return

In [ ]:
def overlap_plan_actual(
        name,
        gdf_plan,
        gdf_actual,
        conditions,
        radius = 10,
        year = None,
        column_year = 'Year_',
        print_map = True,
        test_map = False
    ):
    '''
    - name 
    - gdf_plan - GeoDataFrame of Mobility plan
    - gdf_actual - GeoDataFrame of existing roads, has a column_year
    - conditions - filter specific types of bike/bus lanes
    - radius - radius in meters to widen the line of actual roads
    - year - filter data after construction year
    - column_year - column of construction year
    - print_map - show final map
    - test_map - show interim map
    '''
    print(f'{name} for year after {year}:')
    gdf_plan = gdf_plan.copy()
    gdf_actual = gdf_actual.copy()
    
    if year:
        gdf_actual = gdf_actual[gdf_actual[column_year]>year].copy()
    if conditions['plan']:
        if 'bike' in key:
            gdf_plan = gdf_plan[gdf_plan.BICYCLE_N.isin(
                conditions['plan']
            )].copy()
        if 'bus' in key:
            gdf_plan = gdf_plan[gdf_plan.TRANSIT_N.isin(
                conditions['plan']
            )].copy()
    if conditions['actual']:
            gdf_actual = gdf_actual[gdf_actual.Class.isin(
                conditions['actual']
            )].copy()
    
    print(f'radius = {radius} m')
    # widen linestring to polygon
    gdf_plan_buffer = buffer(
        gdf_plan, 
        radius = radius,
        proj = gdf_plan.geometry.crs
    )
    if test_map:
        display(
            show_map(
                gdf2 = gdf_plan_buffer[['geometry']],
                name2 = 'plan', 
                color2 = 'blue',
                gdf1 = gdf_actual[['geometry']],
                name1 = 'actual', 
                color1 = 'green',
            )
        )
    
    # intersect polygons of actual and planned paths
    gdf_implemented = gpd.overlay(
        gdf_actual, 
        gdf_plan_buffer, 
        how='intersection',
        keep_geom_type=False
    )
    gdf_implemented = get_lengths(gdf_implemented)
    gdf_implemented = gdf_implemented[
        round(gdf_implemented.length_m/(radius*2), 2)>1.01
    ].reset_index(drop=True)
    
    # map of buffered planned (green) and implemented (red) paths
    if print_map:
        display(
            show_map(
                gdf2 = gdf_implemented[['geometry']],
                name2 = 'implemented', 
                color2 = 'red',
                gdf1 = gdf_plan[['geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
        
    print(
        'since', year, 'year:',
        round(get_lengths(gdf_implemented).length_m.sum()/1609.34, 2), 'miles implemented of ',
        round(get_lengths(gdf_plan).length_m.sum()/1609.34, 2), 'miles planned',
        # '\nimplemented lanes records:', gdf_implemented.shape[0],
        # '\nplanned lanes records:', gdf_plan.shape[0]        
    )
    
    percentage = round(
        get_lengths(gdf_implemented).length_m.sum()/
        get_lengths(gdf_plan).length_m.sum()*100, 
        2
    )
    print(f'total {gdf_implemented.shape[0]} records')
    
    return percentage, gdf_implemented

def get_lengths(gdf, column='geometry', proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    return a gdf with a column of linestring length in proj_calc units (m)
    - gdf - GeoDataFrame
    - column - column to be used for length calculation if not "geometry"
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''
    if column:
        gdf.set_geometry(column, inplace=True)
    # convert projection to proj_calc, if default - the units will be in meters
    gdf_m = gdf.to_crs(proj_calc)
    gdf['length_m'] = gdf_m.length
    
    return gdf

In [ ]:
def intersect_gdf_crashes(name, gdf, crashes_gdf, date_col, radius=20):
    
    print('radius for buffer is', radius)
    print('intersecting ', name)
    gdf_buffer = buffer(gdf, radius = radius)
    intersect_crashes = gpd.GeoDataFrame()
    for idx, row in gdf_buffer.iterrows():
        intersection = crashes_gdf[
            crashes_gdf.covered_by(row['geometry'])
        ].copy()
        intersection['install_date'] = row[date_col]
        intersect_crashes = pd.concat([
                intersect_crashes,
                intersection
            ])
        
    return intersect_crashes

# data

## get crashes data

In [ ]:
crashes_raw = pd.read_csv('LAPD_crashes.csv')
display(crashes_raw.head(1))

In [ ]:
cols_dict = {
    'DR Number': 'case_id',
    'Date Occurred': 'collision_date',
    'Address': 'primary_rd',
    'Cross Street': 'secondary_rd',
    'Location': 'location',
    'MO Codes': 'mo_codes',
    'Premise Description': 'loc_description'
}
crashes = crashes_raw.copy().rename(columns = cols_dict)
crashes = crashes[cols_dict.values()]
crashes['year'] = crashes.collision_date.str[-4:].astype(int)
crashes.collision_date = pd.to_datetime(crashes.collision_date).dt.date

new_cols_dict = {
    'injury': ['3024', '3025', '3026'],
    'death': ['3027'],
    #'hit_run': ['3029', '3030'],
    #'doored': ['3603'],
    'veh_ped': ['3003',  '3501'],
    'veh_bike': ['3008'],
    'veh_veh': ['3004']
}
for key, value in new_cols_dict.items():
    crashes[key] = col_from_code(crashes.mo_codes, value)

In [ ]:
crashes.shape[0]

In [ ]:
locations = [
    'STREET', 'SIDEWALK', 'ALLEY', 'DRIVEWAY',
    'MTA BUS', 'BUS STOP'
]

crashes_count = crashes[
    ((crashes.injury!=0)|(crashes.death!=0))&
    ((crashes.veh_ped!=0)|(crashes.veh_bike!=0)|(crashes.veh_veh!=0))&
    (crashes.loc_description.isin(locations))&
    (crashes.year>year_mobility)
].copy().reset_index(drop=True)

crashes_count.location = crashes_count.location.apply(lambda x: get_coords(x))
crashes_count[['loc_y', 'loc_x']] = crashes_count.location.to_list()

In [ ]:
print(crashes.shape[0], crashes_count.shape[0])
display(crashes_count.head())

In [ ]:
#create geodataframe
crashes_gdf = gpd.GeoDataFrame(
    crashes_count[[
        'case_id', 'collision_date', 'year',
        'injury', 'death', 'veh_ped', 'veh_bike', 'veh_veh'
    ]],
    geometry = gpd.points_from_xy(
        x = crashes_count.loc_x,
        y = crashes_count.loc_y,
        crs = 'EPSG:4326')
)

In [ ]:
print(crashes[crashes.injury==1].shape[0])
print(crashes[crashes.death==1].shape[0])

In [ ]:
print(crashes_count[crashes_count.injury==1].shape[0])
print(crashes_count[crashes_count.death==1].shape[0])

In [ ]:
crashes_count[crashes_count.year==2022].shape[0]

## get mobility bikes data with dates of implementation

We can't use previously generated data because the goal was to compare overlap of implemented vs planned against original planned paths, so it was more correct to connect actual data into one buffered shape. Here we need actual data because it has dates when the path was built, so it means we need vice-versa:
1) merge planned data to a buffered shapes
2) overlap it with actual data
3) buffer the result again
4) overlap with the crashes dataset

### Protected/unprotected Bike lanes:

In [ ]:
bike_plan_file = open('../hsla_mobility_plan/Bicycle_Enhanced_Network.geojson')
bike_plan_geo_raw = gpd.read_file(bike_plan_file)
bike_actual_file = open('../hsla_mobility_plan/bike_actual.geojson')
bike_actual_geo_raw = gpd.read_file(bike_actual_file)

In [ ]:
bike_plan_geo = bike_plan_geo_raw.copy()
bike_actual_geo = bike_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bike_plan_geo.shape[0],
    '\nActual bike lanes records:',
    bike_actual_geo.shape[0],
    '\nActual lanes without construction date:',
    bike_actual_geo[bike_actual_geo.Install_Da.isnull()].shape[0]
)
display(bike_plan_geo.head(1))
display(bike_actual_geo.tail(1))

In [ ]:
bike_conditions = {
    'protected bike lane': {'plan': [1], 'actual': [4]},
    'unprotected bike lane': {'plan': [2, 3], 'actual': [2]}
}
results = []
result_gdfs = []

for key in bike_conditions.keys():
    for year_calc in (None, year_mobility):
        percentage, result_gdf = overlap_plan_actual(
            name = key,
            conditions = bike_conditions[key],
            gdf_actual = bike_actual_geo[[
                            'Class',
                            'Year_',
                            'Install_Da',
                            'geometry'
                        ]],
            gdf_plan = bike_plan_geo[[
                            'BICYCLE_N',
                            'geometry'
                       ]],
            print_map = False,
            test_map = False,
            year = year_calc,
            column_year = 'Year_'
        )
        results.append([key, year_calc, percentage])
        result_gdfs.append([key, year_calc, result_gdf])

        print(
            percentage,
            f'% of {key} implemented after {year_calc}',
            '\n'
        )

        print('_______')    

### Bike Paths vs class 1

In [ ]:
bike_plan_paths = open(
    '../hsla_mobility_plan/Bicycle_Enhanced_Network_Paths.geojson'
)
bike_paths_geo_raw = gpd.read_file(bike_plan_paths)
bike_path_geo = bike_paths_geo_raw.copy()
print(bike_path_geo.shape[0])
display(bike_path_geo.head())

In [ ]:
bike_class1_conditions = {
    'class1 bike lane': {'plan': None, 'actual': [1]},
}

for key in bike_class1_conditions.keys():
    for year_calc in (None, year_mobility):
        percentage, result_gdf = overlap_plan_actual(
            name = key,
            gdf_plan = bike_path_geo[['OBJECTID', 'geometry']],
            gdf_actual = bike_actual_geo[[
                                        'Class',
                                        'Year_',
                                        'Install_Da',
                                        'geometry'
                        ]],
            conditions = {'plan': None, 'actual': [1]},
            radius = 120, # take bigger radius beacuse of the data quality
            print_map = False,
            test_map = False,
            year = year_calc,
            column_year = 'Year_'
                )
        print(f'{percentage}% of bike path implemented after {year_calc}')
        results.append([key, year_calc, percentage])
        result_gdfs.append([key, year_calc, result_gdf])
        print('_______')

### NEN vs class 3

In [ ]:
bike_nen_file = open(
    '../hsla_mobility_plan/Neighborhood_Enhanced_Network.geojson'
)
bike_nen_geo_raw = gpd.read_file(bike_nen_file)
bike_nen_geo = bike_nen_geo_raw[['OBJECTID', 'geometry']].copy()
print(bike_nen_geo.shape[0])
display(bike_nen_geo.head())

In [ ]:
bike_nen_conditions = {
    'NEN bike lane': {'plan': None, 'actual': [3]},
}

for key in bike_nen_conditions.keys():
    for year_calc in (None, year_mobility):
        percentage, result_gdf = overlap_plan_actual(
            name = key,
            conditions = bike_nen_conditions[key],
            gdf_actual = bike_actual_geo[[
                                        'Class',
                                        'Year_',
                                        'Install_Da',
                                        'geometry'
                        ]],
            gdf_plan = bike_nen_geo,
            print_map = False,
            year = year_calc,
            column_year = 'Year_',
            test_map = False
        )
        results.append([key, year_calc, percentage])
        result_gdfs.append([key, year_calc, result_gdf])

        print(
            percentage,
            f'% of {key} implemented after {year_calc}',
            '\n'
        )

        print('_______')   

### all bike mobility implemented

In [ ]:
mobility_implemented_bike = gpd.GeoDataFrame()
mobility_implemented_2015_bike = gpd.GeoDataFrame()
for df in result_gdfs:
    gdf = df[2].copy()
    gdf['name'] = df[0].replace(' ', '_') + '_implemented'
    yr = df[1]
    print(f'{df[0]} after {df[1]}, {gdf.shape[0]} records')
    display(gdf.head(1))
    if yr:
        gdf['name'] = gdf['name'] + f'_since_{yr}'
        if mobility_implemented_2015_bike.shape[0]==0:
            mobility_implemented_2015_bike = gdf[['name', 'Install_Da', 'geometry']].copy()
        else:
            mobility_implemented_2015_bike = pd.concat([
                mobility_implemented_2015_bike,
                gdf[['name', 'Install_Da', 'geometry']]]
            )
    else:
        if mobility_implemented_bike.shape[0]==0:
            mobility_implemented_bike = gdf[['name', 'Install_Da', 'geometry']].copy()
        else:
            mobility_implemented_bike = pd.concat([
                mobility_implemented_bike,
                gdf[['name', 'Install_Da', 'geometry']]]
            )
mobility_implemented_bike.reset_index(inplace=True, drop=True)
mobility_implemented_bike.Install_Da = pd.to_datetime(
    mobility_implemented_bike.Install_Da
).dt.date
display(mobility_implemented_bike.head(1))
mobility_implemented_2015_bike.reset_index(inplace=True, drop=True)
mobility_implemented_2015_bike.Install_Da = pd.to_datetime(
    mobility_implemented_2015_bike.Install_Da
).dt.date
display(mobility_implemented_2015_bike.head(1))

## get mobility buses data with dates of implementation

In [ ]:
bus_plan_file = open('../hsla_mobility_plan/Transit_Enhanced_Network.geojson')
bus_plan_geo_raw = gpd.read_file(bus_plan_file)
bus_actual_file = open('../hsla_mobility_plan/bus_only_lanes_sat.geojson')
bus_actual_geo_raw = gpd.read_file(bus_actual_file)

In [ ]:
bus_plan_geo = bus_plan_geo_raw.copy()
bus_actual_geo = bus_actual_geo_raw.copy()

print(
    'Mobility plan records:',
    bus_plan_geo.shape[0],
    '\nActual bus lanes records:',
    bus_actual_geo.shape[0]
)
display(bus_plan_geo.head(1))
display(bus_actual_geo.tail(1))

In [ ]:
bus_conditions = {
    'bus lane': {'plan': [3, 2], 'actual': None},
}
bus_results = []
bus_result_gdfs = []
for key in bus_conditions.keys():
    for year_calc in (None, year_mobility):
        percentage, result_gdf = overlap_plan_actual(
            name = key,
            conditions = bus_conditions[key],
            radius = 20,
            gdf_actual = bus_actual_geo[[
                            'Name',
                            'geometry',
                            'Year_'
                        ]].copy(),
            gdf_plan = bus_plan_geo,
            print_map = False,
            test_map = False,
            year = year_calc,
            column_year ='Year_' 
        )
        bus_results.append([key, year_calc, percentage])
        bus_result_gdfs.append([key, year_calc, result_gdf])

In [ ]:
mobility_implemented_bus = gpd.GeoDataFrame()
mobility_implemented_2015_bus = gpd.GeoDataFrame()
for df in bus_result_gdfs:
    gdf = df[2].copy()
    gdf['name'] = df[0].replace(' ', '_') + '_implemented'
    print(f'{df[0]} after {df[1]}')
    display(gdf.head(1))
    if yr:
        gdf['name'] = gdf['name'] + f'_since_{yr}'
        if mobility_implemented_2015_bus.shape[0]==0:
            mobility_implemented_2015_bus = gdf[['name', 'Year_', 'geometry']].copy()
        else:
            mobility_implemented_2015_bus = pd.concat([
                mobility_implemented_2015_bus,
                gdf[['name', 'Year_', 'geometry']]]
            )
    else:
        if mobility_implemented_bus.shape[0]==0:
            mobility_implemented_bus = gdf[['name', 'Year_', 'geometry']].copy()
        else:
            mobility_implemented_bus = pd.concat([
                mobility_implemented_bus,
                gdf[['name', 'Year_', 'geometry']]]
            )
mobility_implemented_bus.reset_index(inplace=True, drop=True)
display(mobility_implemented_bus.head(1))
mobility_implemented_2015_bus.reset_index(inplace=True, drop=True)
display(mobility_implemented_2015_bus.head(1))

# get stats

### bike lanes

In [ ]:
crashes_results = []
crashes_result_gdfs = []

for radius in [20]:#, 30, 40]:
    impl_crashes = intersect_gdf_crashes(
        name = 'implemented_crashes',
        gdf = mobility_implemented_2015_bike,
        crashes_gdf = crashes_gdf,
        date_col = 'Install_Da',
        radius = radius
    )
    impl_preventable = impl_crashes[
          impl_crashes.collision_date<impl_crashes.install_date
      ].copy().reset_index(drop=True)
    num_preventable = impl_preventable.shape[0]
    num_impl_crashes = impl_crashes.shape[0]
    print(f'{num_preventable} preventable of {num_impl_crashes} crashes')
    crashes_results.append(['bike', radius, num_impl_crashes, num_preventable])
    crashes_result_gdfs.append(['bike', radius, impl_preventable])

In [ ]:
impl_preventable.head(2)

In [ ]:
impl_crashes[impl_crashes.year==2022].shape

In [ ]:
# impl_crashes = crashes_result_gdfs[2][2]
# m = show_maps({
#     'crashes': impl_crashes[['geometry']],
#     'implemented': mobility_implemented_bike[['geometry']]
# })
# m

### bus lanes

In [ ]:
for radius in [20]:#, 30, 40]:
    impl_crashes = intersect_gdf_crashes(
        name = 'implemented_crashes',
        gdf = mobility_implemented_2015_bus,
        crashes_gdf = crashes_gdf,
        date_col = 'Year_',
        radius = radius
    )
    impl_preventable = impl_crashes[
          impl_crashes.year<impl_crashes.install_date
      ].copy().reset_index(drop=True)
    num_preventable = impl_preventable.shape[0]
    num_impl_crashes = impl_crashes.shape[0]
    print(f'{num_preventable} preventable of {num_impl_crashes} crashes')
    crashes_results.append(['bus', radius, num_impl_crashes, num_preventable])
    crashes_result_gdfs.append(['bus', radius, impl_preventable])

In [ ]:
impl_crashes[impl_crashes.year==2022].shape

In [ ]:
impl_crashes.head()

In [ ]:
impl_crashes[impl_crashes.year==2022].shape

In [ ]:
crashes_results

## save results

In [ ]:
crashes_results_df = pd.DataFrame(
    columns=[
        'transport type',
        'radus_m',
        'crashes_mobility',
        'preventable_crashes'
    ],
    data = crashes_results
)
display(crashes_results_df)

In [ ]:
# crashes_results_df.to_csv(f'preventable_crashes_{today}.csv', index=False)
# 
# for df in crashes_result_gdfs:
#     gdf = df[2].copy()
#     gdf.collision_date = pd.to_datetime(gdf.collision_date).dt.strftime('%Y/%m/%d')
#     gdf.install_date = pd.to_datetime(gdf.install_date).dt.strftime('%Y/%m/%d')
#     gdf.to_file(
#         f'preventable_crashes_{df[0]}_lanes_buffer{df[1]}m_{today}.geojson',
#         driver='GeoJSON'
#     )

# troubleshoot

In [ ]:
impl_crashes_2022 = open('impl_crashes_2022.geojson')
impl_crashes_2022_tims = gpd.read_file(impl_crashes_2022)
display(impl_crashes_2022_tims.head())

preventable_crashes_2022_lapd = crashes_result_gdfs[0][2]
preventable_crashes_2022_lapd = preventable_crashes_2022_lapd[
        preventable_crashes_2022_lapd.year==2022
    ].copy().reset_index(drop=True)
display(preventable_crashes_2022_lapd.head())

In [ ]:
mobility_implemented_bike.head()

In [ ]:
m = show_maps({
    'lapd': preventable_crashes_2022_lapd[['geometry']],
    'tims': impl_crashes_2022_tims[['geometry']],
    'implemented bike': mobility_implemented_bike[['geometry']],
    'implemented bus': mobility_implemented_bus[['geometry']]
})
m

In [ ]:
m_implemented_tims_file = open('../mobility_implemented.geojson')
m_implemented_tims = gpd.read_file(m_implemented_tims_file)
display(m_implemented_tims.head())

In [ ]:
m = show_maps({
    'tims': m_implemented_tims[['geometry']],
    'implemented bike': mobility_implemented_bike[['geometry']],
    'implemented bus': mobility_implemented_bus[['geometry']]
})
m